In [ ]:
!pip install opencv-python numpy imutils
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os


In [ ]:
def preprocess_strong(img):
    # 1. Resize (keeps processing stable)
    h, w = img.shape[:2]
    scale = 900 / max(h, w)
    img = cv2.resize(img, None, fx=scale, fy=scale)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 2. Remove lighting variations (CLAHE)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    gray_eq = clahe.apply(gray)

    # 3. Light noise reduction
    blur = cv2.GaussianBlur(gray_eq, (5,5), 0)

    # 4. Hybrid threshold (works for dark + bright)
    otsu = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    adap = cv2.adaptiveThreshold(blur, 255,
                                 cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                 cv2.THRESH_BINARY_INV,
                                 11, 2)

    # Combine thresholds (more robust)
    thresh = cv2.bitwise_or(otsu, adap)

    # 5. Morphology to strengthen grid lines
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    strong = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

    return img, strong


In [ ]:
def find_outer_frame(binary):
    # Try contour approach
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        c = max(contours, key=cv2.contourArea)
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)

        if len(approx) == 4:
            return approx.reshape(4,2)

    # Fallback: detect via Hough lines
    edges = cv2.Canny(binary, 50, 150, apertureSize=3)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=120,
                            minLineLength=200, maxLineGap=20)
    if lines is None:
        return None

    pts = []
    for l in lines:
        x1,y1,x2,y2 = l[0]
        pts.append((x1,y1))
        pts.append((x2,y2))

    pts = np.array(pts)
    rect = cv2.boundingRect(pts.reshape(-1,1,2))
    x,y,w,h = rect
    corners = np.array([[x,y],[x+w,y],[x+w,y+h],[x,y+h]], dtype=np.float32)
    return corners


In [ ]:
def order_points(pts):
    pts = pts[np.argsort(pts[:,0])]    # sort by x
    left = pts[:2]
    right = pts[2:]

    tl, bl = left[np.argsort(left[:,1])]
    tr, br = right[np.argsort(right[:,1])]
    return np.array([tl, tr, br, bl], dtype=np.float32)

def warp_grid(img, pts, size=900):
    pts = order_points(pts)
    dst = np.array([[0,0],[size,0],[size,size],[0,size]], dtype=np.float32)
    M = cv2.getPerspectiveTransform(pts, dst)
    return cv2.warpPerspective(img, M, (size,size))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2



input_folder = "/content/drive/MyDrive/ProjectTestCases"


output_folder = os.path.join(input_folder, "output_processed")
os.makedirs(output_folder, exist_ok=True)


for filename in os.listdir(input_folder):
    if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    
    img_path = os.path.join(input_folder, filename)
    img = cv2.imread(img_path)

    
    if img is None:
        print(f"FAILED to read image: {filename}")
        continue

    
    resized, binimg = preprocess_strong(img)

    
    corners = find_outer_frame(binimg)
    if corners is None:
        print("FAILED: Frame not found in", filename)
        continue

   
    dbg = resized.copy()
    for x,y in corners:
        
        cv2.circle(dbg, (int(x),int(y)), 10, (0,0,255), -1)

   
    warped = warp_grid(resized, corners)

    
    cv2.imwrite(os.path.join(output_folder, f"{filename}_bin.png"), binimg)
    cv2.imwrite(os.path.join(output_folder, f"{filename}_corners.png"), dbg)
    cv2.imwrite(os.path.join(output_folder, f"{filename}_warp.png"), warped)

    print("OK:", filename)

OK: 09.jpg
OK: 10.jpg
OK: 13.jpg
OK: 11.jpg
OK: 12.jpg
OK: 14.jpg
OK: 15.jpg
OK: 16.jpg
OK: 01.jpg
OK: 02.jpg
OK: 03.jpg
OK: 04.jpg
OK: 05.jpg
OK: 06.jpg
OK: 08.jpg
OK: 07.jpg
